# Bischöfe
## Datenimport für Wiagweb 1

Angleichung an Domherren für Klosterorte

- [Ämter](#Ämter)

2021-11-26

In [1]:
# wds_path="C:\\Users\\Georg\\Documents\\projekte\\WiagDataSetup.jl"
wds_path="..\\.."

"..\\.."

In [2]:
cd(wds_path)

In [3]:
pwd()

"C:\\Users\\georg\\Documents\\projekte\\WiagDataSetup.jl"

In [4]:
using Pkg

In [5]:
Pkg.activate(".")

In [6]:
using Revise

In [7]:
using WiagDataSetup

┌ Info: Precompiling WiagDataSetup [522c5ebb-a018-4020-8ed4-420cb1a9f084]
└ @ Base loading.jl:1317


In [8]:
using MySQL

In [9]:
using DataFrames

In [10]:
Wds = WiagDataSetup

WiagDataSetup

In [11]:
Wds.setDBWIAG(user="georg", db="wiag_in")

Passwort für User georg
stdin> topf&deckel


MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag_in")

## Ämter

Setze `numdate_start` und `numdate_end`

In [12]:
sql = "SELECT * FROM office"
df_oc = Wds.sql_df(sql);

In [14]:
date_start(date) = Wds.parsemaybe(date, :lower)
transform!(df_oc, :date_start => ByRow(date_start) => :numdate_start);

┌ Warning: Could not parse 
│   s = fränkische Zeit ?
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1111


In [15]:
date_end(date) = Wds.parsemaybe(date, :upper)
transform!(df_oc, :date_end => ByRow(date_end) => :numdate_end);

In [17]:
table_name="office"
Wds.filltable!(table_name, df_oc, clear_table = true)

┌ Info: Rows inserted: 7329
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1202


7329

### `office.location_show`

In [52]:
sql = "SELECT * FROM monastery_location"
df_mon_loc = Wds.sql_df(sql);

In [53]:
transform!(df_mon_loc, :location_begin_tpq => ByRow(date_start) => :loc_start);

In [54]:
transform!(df_mon_loc, :location_end_tpq => ByRow(date_end) => :loc_end);

In [55]:
parseId(id) = parse(Int, id)
transform!(df_mon_loc, :wiagid_monastery => ByRow(parseId) => :wiagid_monastery);

In [37]:
df_mon_loc[17:22, [:wiagid_monastery, :loc_start, :loc_end]]

,wiagid_monastery,loc_start,loc_end
,String?,Int64,Int64
1,81,1027,1803
2,93,1005,1803
3,95,1020,1810
4,99,1054,1810
5,100,1133,1803
6,101,1196,1810


In [56]:
in_epic_mon(id) = id in (4623, 20165, 791)
findall(in_epic_mon, df_mon_loc.wiagid_monastery)

4-element Vector{Int64}:
 3
 7
 8
 9

Übernimm zunächst den Wert aus `location`

In [21]:
size(df_oc)

(7329, 19)

In [57]:
df_oc[:, :location_show] .= df_oc[:, :location];

In [58]:
size(filter(:location_show => ismissing, df_oc))

(7329, 19)

Suche für den Rest eindeutige Einträge in `df_mon_loc`

In [30]:
"""
    findvalid(cols, df)

return first value in `df[:, cols]` which is not missing
"""
function findvalid(cols, df)
    val = missing
    for row in eachrow(df), col in cols
        val = row[col]
        if !ismissing(val)
            break
        end
    end
    return val
end

findvalid (generic function with 1 method)

In [38]:
"""
    match_dates(loc_start, loc_end, date_start, date_end)

return `true` if `[:date_start, :date_end]` id not outside of `[:loc_start], [:loc_end]`
"""
function match_dates(loc_start, loc_end, date_start, date_end)
    if !ismissing(loc_start) && !ismissing(date_end) && loc_start > date_end
        return false
    end
    if !ismissing(loc_end) && !ismissing(date_start) && loc_end < date_start
        return false
    end
    return true    
end

match_dates

Setze `location_show` mit Angaben aus `monastery_location`

In [63]:
n_match = 0
for row in eachrow(df_oc)
    id_monastery = row[:id_monastery]
    if !ismissing(row[:location_show]) || ismissing(id_monastery)
        continue
    end
    location_show = missing
    df_match = filter(:wiagid_monastery => isequal(id_monastery), df_mon_loc)    
    if size(df_match, 1) > 1
        ff(loc_start, loc_end) = match_dates(loc_start, loc_end, row[:numdate_start], row[:numdate_end])
        df_match = filter([:loc_start, :loc_end] => ff, df_match)
    end
    location_show = findvalid([:location_name, :place_name], df_match)
    if !ismissing(location_show)
        row[:location_show] = location_show
        n_match += 1
    end
end 
n_match

0

In [64]:
ix_mg = findall(ismissing, df_oc.location_show);
length(ix_mg)

7287

In [66]:
df_oc[ix_mg, :location_show] .= df_oc[ix_mg, :diocese];

In [67]:
size(filter(:location_show => ismissing, df_oc))

(84, 19)

In [69]:
table_name = "office"
Wds.filltable!(table_name, df_oc, clear_table = true)

┌ Info: Rows inserted: 7329
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1202


7329